In [1]:
%load_ext autoreload
%autoreload 2

In [103]:
import pandas as pd 
import numpy as np
import json
import copy
import time
from datetime import datetime 

from api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap, threading
from preproc import filters, make_dfs as df

In [3]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2016, 8, 9), datetime(2022, 8, 9)]
    )

In [4]:
#get tickers that are valid across the past six years
tker_dict = val.get_tkers(settings)
valid_lst = val.get_valid_tkers(tker_dict, pretty_print = False)

In [6]:
#filtering the tickers we want for analysis 
raw_df = pd.DataFrame(valid_lst, columns = ["ticker", "name", "exchange", "type", "date", "delist", "status"])
type_filt = "Stock" 
exchange_filt = "NYSE"
raw_df = raw_df.query("type == @type_filt")
raw_df = raw_df.query("exchange == @exchange_filt")
display(raw_df)

#list of tickers to work on for now
ticker_lst = raw_df["ticker"].tolist()
#test_lst = raw_df["ticker"].tolist()[:1000]

,ticker,name,exchange,type,date,delist,status
0,DAN,Dana Inc,NYSE,Stock,2008-01-02,null,Active
8,PEAK,Healthpeak Properties Inc,NYSE,Stock,1987-11-05,null,Active
16,MX,MagnaChip Semiconductor Corp,NYSE,Stock,2011-03-11,null,Active
17,SCX,L.S. Starrett Co. - Class A,NYSE,Stock,1987-12-30,null,Active
18,SLG-P-I,SL Green Realty Corporation Preferred Series I,NYSE,Stock,2002-07-01,null,Active
...,...,...,...,...,...,...,...
5574,CMP,Compass Minerals International Inc,NYSE,Stock,2003-12-12,null,Active
5581,JNPR,Juniper Networks Inc,NYSE,Stock,1999-06-25,null,Active
5583,BAP,Credicorp Ltd,NYSE,Stock,1995-10-25,null,Active
5584,ENB,Enbridge Inc,NYSE,Stock,1990-03-27,null,Active


In [7]:
#get cofi and filter out bad tickers 
cofi_lst = ['MarketCapitalization', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta']
hit_lst = copy.deepcopy(ticker_lst)
time_it = 3
ticker_info_data = {}
while len(hit_lst) > 0 and time_it > 2:
    ticker_info_url = wrap.get_general_url(settings, ticker_lst = hit_lst, output = "lst", datatype = "csv", function = "OVERVIEW")
    hit_lst, time_it, data = threading.thrd_json_data(ticker_info_url, key_lst = cofi_lst, wrapper_function = wrap.get_json_data)
    ticker_info_data.update(data)
    print(hit_lst)
    time.sleep(10)
with open(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\NYSE_cofi.json', 'w') as fp:
    json.dump(ticker_info_data, fp)

Total time elapsed: 121.25703263282776 seconds
['ACC', 'RBC', 'ELME', 'KN', 'TM', 'NVO', 'WSR', 'AES', 'HBM', 'CPA', 'E', 'BP', 'BGS', 'NIC', 'STAR-P-I', 'PEI-P-B', 'CNC', 'AIG', 'AEE', 'UNF', 'SNX', 'W', 'HPQ', 'GCO', 'PG', 'UTL', 'MMI', 'TSN', 'HSBC', 'ATI', 'OLP', 'WIT', 'HSY', 'CRK', 'BF-A', 'CTRA', 'GES', 'XIN', 'RENN', 'CAH', 'WWE', 'JCI', 'FENG', 'SLCA', 'CPK', 'MPX', 'WLKP', 'COTY', 'CUBI-P-E', 'FIS', 'FIX', 'CFR', 'TR', 'FCF', 'TRQ', 'CFG', 'GAB-P-G', 'APTV', 'EBR-B', 'TNK', 'USDP', 'WDS', 'RNG', 'DLR', 'DBD', 'KB', 'STOR', 'FRO', 'TPX', 'LC', 'PAYC', 'PLD', 'ASPN', 'RY-P-T', 'AGI', 'HES', 'KNL', 'DDD', 'MAA', 'BCO', 'LMT', 'ENZ', 'GD', 'WNS', 'AAP', 'MWA', 'PPL', 'EGO', 'BVH', 'STT', 'MMS', 'PNW', 'MUFG', 'CSV', 'HLF', 'KNOP', 'ASH', 'CTA-P-B', 'NEE', 'MHNC', 'BF-B', 'WHG', 'RELX', 'NGVT', 'ALE', 'GBLI', 'SRE', 'OLN', 'DAL', 'OIS', 'H', 'SBOW', 'BRK-B', 'UGI', 'GGG', 'SCHW', 'AHL-P-C', 'BC', 'JWN', 'SA', 'R', 'CPRI', 'UDR', 'BML-P-L', 'WCC', 'AORT', 'TDW', 'CWEN-A', 'HE', 'KR

In [11]:
#filter out tickers with missing data
with open(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\NYSE_cofi.json', 'r') as fp:
    ticker_info_data = json.load(fp)
cofi_filter, filtered_cofi_dict = filters.cofi_filter(ticker_info_data)
clean_df = raw_df.query("ticker != @cofi_filter")
display(clean_df)

,ticker,name,exchange,type,date,delist,status
0,DAN,Dana Inc,NYSE,Stock,2008-01-02,null,Active
8,PEAK,Healthpeak Properties Inc,NYSE,Stock,1987-11-05,null,Active
16,MX,MagnaChip Semiconductor Corp,NYSE,Stock,2011-03-11,null,Active
19,CRS,Carpenter Technology Corp,NYSE,Stock,1987-11-05,null,Active
28,PWR,Quanta Services Inc,NYSE,Stock,1998-02-12,null,Active
...,...,...,...,...,...,...,...
5574,CMP,Compass Minerals International Inc,NYSE,Stock,2003-12-12,null,Active
5581,JNPR,Juniper Networks Inc,NYSE,Stock,1999-06-25,null,Active
5583,BAP,Credicorp Ltd,NYSE,Stock,1995-10-25,null,Active
5584,ENB,Enbridge Inc,NYSE,Stock,1990-03-27,null,Active


In [60]:
temp_lst = clean_df["ticker"].tolist()

In [65]:
#get daily data 
hit_lst_2 = copy.deepcopy(temp_lst)
time_it_2 = 3
daily_data = {}
while len(hit_lst_2) > 0 and time_it_2 > 2:
    daily_urls = wrap.get_general_url(settings, datatype = "csv", ticker_lst = hit_lst_2, output = "lst", function = "TIME_SERIES_DAILY_ADJUSTED")
    hit_lst_2, time_it_2, data = threading.thrd_csv_data(daily_urls)
    daily_data.update(data)
    print(hit_lst_2)
    time.sleep(15)
with open(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\NYSE_daily_data.json', 'w') as fp:
    json.dump(daily_data, fp)

'ELV' generated an exception: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Total time elapsed: 107.20370101928711 seconds
['TMX', 'HLT', 'LXU', 'BA', 'ORA', 'CBZ', 'SKT', 'DOOR', 'AVD', 'PRA', 'OI', 'PARR', 'NYT', 'RBC', 'FNB', 'KIM', 'SAP', 'JLL', 'NOK', 'LL', 'KAI', 'ITW', 'USAC', 'CHMI', 'LSI', 'VPG', 'TGI', 'ROP', 'TUP', 'ATGE', 'DD', 'ORAN', 'DEA', 'BRC', 'ARI', 'BMI', 'REX', 'NGS', 'ATTO', 'TSLX', 'CE', 'HI', 'HRI', 'BCC', 'PHI', 'SQM', 'EMN', 'ELS', 'PRO', 'EQT', 'VFC', 'ELME', 'BDN', 'DOV', 'HUN', 'ST', 'KN', 'SHO', 'TTI', 'TM', 'WSR', 'CPA', 'AES', 'E', 'BP', 'NIC', 'BGS', 'AIG', 'CNC', 'AEE', 'UNF', 'NVO', 'SNX', 'HBM', 'W', 'UTL', 'GCO', 'HPQ', 'MMI', 'HSBC', 'PG', 'TSN', 'ATI', 'OLP', 'WIT', 'HSY', 'WWE', 'CAH', 'CRK', 'RENN', 'GES', 'XIN', 'CTRA', 'JCI', 'FENG', 'SLCA', 'CPK', 'MPX', 'FIS', 'WLKP', 'CFR', 'TR', 'FIX', 'CFG', 'TRQ', 'APTV', 'FCF', 'COTY', 'EBR-B', 'TNK', 'USDP', 'RNG

In [101]:
start = settings.date_range[0] 
end = settings.date_range[1]
data_dict = {i: [[j[0], float(j[5]), int(j[6])] for j in daily_data[i][1:]] for i in daily_data.keys()}
data_key = list(data_dict.keys())
for i in data_key: 
    data_dict[i] = pd.DataFrame(data_dict[i][1+1:], columns = [daily_data[i][0][0], daily_data[i][0][5], daily_data[i][0][6]])
    data_dict[i]["timestamp"] = pd.to_datetime(data_dict[i]["timestamp"])
    data_dict[i] = data_dict[i].query("@start <= timestamp <= @end")
    if len(data_dict[i]) != 1511: 
        print(i, len(data_dict[i]))
        data_dict.pop(i)

FERG 1499
ACC 1509
BUR 1404
TFII 1506
SDRL 973
DINO 1507
EFXT 1497
NIC 1501
EBR-B 1494
SBOW 1510
HIL 1467
ENOV 1510
NE 479
LAB 87
WFG 1505
AENZ 1481
IVT 1486
NEE-P-R 771


In [104]:
price_pivot = df.pivot_data(data_dict, key = "adjusted_close")
vol_pivot = df.pivot_data(data_dict, key = "volume")

price_pivot.to_csv(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\price_pca.csv')
vol_pivot.to_csv(r'C:\Users\ELIZABETH CHENG\MLAP_Proj\pairs_selection\reader\vol_pca.csv')

In [80]:
data_filt, debug = filters.daily_filter(settings, daily_data)

here


In [70]:
from preproc import make_dfs as df

price_pivot = df.pivot_data(data_filt, key = "adjusted_close")
vol_pivot = df.pivot_data(data_filt, key = "volume")

display(vol_pivot)

""
0
UAN
NPO
PWR
MX
DAN
...
AGRO
PEN
ENB


In [23]:
daily_data.keys()

dict_keys(['PLNT', 'UAN', 'DAN', 'AVY', 'NPO', 'PWR', 'SWN', 'CRS', 'VMW', 'MX', 'TNET', 'VTR', 'SPXC', 'AVNS', 'IRM', 'VRTV', 'PFS', 'TEN', 'PEAK', 'FCPT', 'OKE', 'AXTA', 'DLX', 'UNVR', 'WPC', 'HMY', 'PVH', 'MD', 'APO', 'PBH', 'CRI', 'CVX', 'RLJ', 'DY', 'COP', 'RDN', 'CMS', 'NX', 'Y', 'VET', 'GOLD', 'AVB', 'GMRE', 'AON', 'EVTC', 'ARW', 'BW', 'DKS', 'GCI', 'AXL', 'MSM', 'APAM', 'NS', 'D', 'CCO', 'WAL', 'RF', 'UL', 'ACRE', 'BPT', 'LII', 'EGHT', 'SPG', 'GWRE', 'ANET', 'MTG', 'CAT', 'CSTM', 'FSS', 'BOOT', 'NC', 'VLRS', 'HZN', 'JPM', 'UBS', 'USNA', 'GDOT', 'HRTG', 'KAR', 'HRL', 'TSE', 'TMHC', 'WBS', 'SNDA', 'BEP', 'HUBB', 'BRT', 'DK', 'EAT', 'FTK', 'WY', 'K', 'SID', 'TKR', 'LYG', 'ICL', 'ABB', 'HEI', 'LPX', 'FERG'])

In [24]:
testing = filters.daily_filter(settings, daily_data)
testing["DLX"]

,timestamp,adjusted_close,volume
0,2022-08-09,22.452614,228192
1,2022-08-08,23.034902,235400
2,2022-08-05,23.261895,256031
3,2022-08-04,24.989020,181881
4,2022-08-03,25.048235,180165
...,...,...,...
1494,2016-08-15,56.423068,160122
1495,2016-08-12,55.964480,118097
1496,2016-08-11,56.181267,327273
1497,2016-08-10,56.723235,153910


In [225]:
from preproc import make_dfs as df

price_pivot = df.pivot_data(testing, key = "adjusted_close")
vol_pivot = df.pivot_data(testing, key = "volume")

display(vol_pivot)

,1,2,3,4,5,6,7,8,9,10,...,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495
0,,,,,,,,,,,,,,,,,,,,,
SUPV,186158,312678,392690,335482,214644,528477,358797,840771,450436,239313,...,195692,85849,320715,436921,43803,110219,278999,228546,74152,342065
GS-P-J,46442,42442,82078,66733,47322,55837,98564,304376,97451,375893,...,73827,53553,66223,82248,63611,79565,101893,79444,148313,153219
FPI,287750,392494,278950,520773,428990,562345,1843402,672827,561023,527444,...,98178,108093,159272,59952,65814,156703,140746,58422,50396,46923
SO,3149574,4259566,4136366,5241990,4436915,4795010,4324585,7830140,5840971,6341737,...,3655715,4830885,6552814,7694959,21767980,5030157,4124547,3190805,5836874,4201527
TKC,515737,540460,595103,275476,619683,438066,577029,581867,533888,644573,...,215640,267690,144818,234982,134990,155717,135193,172215,293123,179163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RDY,130758,133900,128473,94619,247877,209064,153202,152081,326072,159561,...,270752,81019,126623,170494,589004,108743,88848,143069,212704,290566
SR,318738,289608,296278,283317,224674,195115,269958,308374,211292,236569,...,117515,119042,173809,203301,136745,258018,220969,135428,189415,165287
CPA,300127,225032,243926,792229,353649,193275,237111,285584,263296,216715,...,397717,330063,618947,526402,965264,685789,435418,598943,398252,453241


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#PCA logic flow 

#Step 1: get the standardised features
x = df (with the features from the dataframe)
# Standardizing the features
x = StandardScaler().fit_transform(x)

#Step 2: feeed standardadised features into the PCA
#We can experiment with different PCA n later? 
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ["pc1", "pc2"])

#Step 3: Check if explained variance is good enough
print(pca.explained_variance_ratio_)

#Step 4: 
Visualisation for slides idk 

In [117]:
daily_data.keys()

dict_keys(['SUPV', 'GS-P-J', 'BGSF', 'FPI', 'JBT', 'TKC', 'DRH', 'GHL', 'UNFI', 'SRE', 'MCS', 'SO', 'BML-P-H', 'USB-P-A', 'REX', 'EGP', 'SJW', 'MTRN', 'ETD', 'MUFG', 'PB', 'GJR', 'GDOT', 'TFC', 'SNA', 'RGA', 'F', 'IBM', 'LTC', 'YRD', 'BC', 'BSBR', 'GSK', 'MTN', 'CHU', 'NUS', 'RZB', 'PII', 'IDA', 'LN', 'GNL', 'J', 'AWI', 'PRG', 'SOL', 'DRD', 'FNF', 'WTI', 'IRS', 'C'])

KeyError: "None of ['timestamp'] are in the columns"

In [58]:
import requests, json

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'IBM', '3. Last Refreshed': '2022-10-27', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2022-10-27': {'1. open': '135.55', '2. high': '136.4', '3. low': '134.445', '4. close': '134.77', '5. adjusted close': '134.77', '6. volume': '3993168', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-26': {'1. open': '133.72', '2. high': '135.863', '3. low': '132.81', '4. close': '135.01', '5. adjusted close': '135.01', '6. volume': '5140023', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-25': {'1. open': '132.0', '2. high': '133.3', '3. low': '131.3', '4. close': '132.93', '5. adjusted close': '132.93', '6. volume': '5957623', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2022-10-24': {'1. open': '130.9', '2. high': '133.11', '3. low': '129.85', '4. close': '132.69', '5. adjusted close': '132

In [218]:
import requests, json
url = 'https://www.alphavantage.co/query?function=OVERVIEW&output=json&symbol=IBM&apikey=MD27QTTVAK3AJBUQ'
r = requests.get(url)
data = r.json()
print(data.keys())

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])
